# Design fashion using AI (Diseña la moda con AI)

Pupose of this notebook is fine tunning a difussion model in order to use fashion related promts, so it could generate fashion detailed images

He is also well explained: https://huggingface.co/blog/stable_diffusion
keras tutorial and explanation can be found here: https://keras.io/examples/generative/finetune_stable_diffusion/,
The keras tutorial is build below: https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/train_text_to_image.py

The last step script is the one that we have explained here but step by step

In [1]:
import sys
sys.path.append('/data/diffusers-t3chfest/src/')

In [2]:
from diffusers import (AutoencoderKL, DDPMScheduler, 
                       StableDiffusionPipeline, 
                       UNet2DConditionModel)
from diffusers.optimization import get_scheduler
from diffusers.training_utils import EMAModel
from transformers import CLIPTextModel, CLIPTokenizer
from accelerate import Accelerator
from torch import optim

## Load pretrained models and define its params

### Define inputs and pre-trained models:

In [15]:
# pretrained model
model_id = 'stabilityai/stable-diffusion-2-1-base'

# accelerator params
gradient_accumulation_steps = 1
mixed_precision = 'no' # 'fp16' # for gpu

# optimiezer params
learning_rate = 1e-4

### Acelerate

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
https://stackoverflow.com/questions/70461130/clarification-about-gradient-accumulation

In [4]:
accelerator = Accelerator(
    gradient_accumulation_steps=gradient_accumulation_steps,
    cpu=True,
    mixed_precision=mixed_precision
)

Define stable diffusion pipeline

In [5]:
# pipe = StableDiffusionPipeline.from_pretrained(model_id).to(device)
# here a lot of things are going to be dowloaded

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

NameError: name 'device' is not defined

### Load autoencoders

Load the autoencoder model which will be used to decode the latents into image space. 

The autoencoder used in Stable Diffusion has a reduction factor of 8. This means that an image of shape (3, 512, 512) becomes (3, 64, 64) in latent space, which requires 8 × 8 = 64 times less memory.

In [6]:
autoencoder = AutoencoderKL.from_pretrained(model_id, subfolder="vae")

### Load text encoder and tokenizer

Load the tokenizer and text encoder to tokenize and encode the text. 

In [7]:
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.11.self_attn.v_proj.weight', 'vision_model.encoder.layers.6.self_attn.k_proj.bias', 'vision_model.encoder.layers.6.layer_norm1.weight', 'vision_model.encoder.layers.6.self_attn.k_proj.weight', 'vision_model.encoder.layers.10.mlp.fc1.bias', 'vision_model.encoder.layers.2.mlp.fc1.bias', 'vision_model.encoder.layers.3.self_attn.out_proj.weight', 'vision_model.encoder.layers.15.self_attn.q_proj.bias', 'vision_model.encoder.layers.17.self_attn.k_proj.weight', 'vision_model.encoder.layers.0.self_attn.k_proj.bias', 'vision_model.encoder.layers.9.layer_norm2.weight', 'vision_model.encoder.layers.1.self_attn.k_proj.weight', 'vision_model.encoder.layers.14.self_attn.q_proj.weight', 'vision_model.encoder.layers.1.self_attn.out_proj.bias', 'vision_model.encoder.layers.21.mlp.fc2.weight', 'vision_model.encoder.layers.22.mlp.fc2.bias', 'vision_model.encod

### U-NET model

The UNet model for generating the latents.

In [8]:
unet = UNet2DConditionModel.from_pretrained(model_id, subfolder="unet")

### Scheduler

Denoising diffusion probabilistic models (DDPMs) explores the connections between denoising score matching and Langevin dynamics sampling.

In [9]:
noise_scheduler = DDPMScheduler.from_pretrained(model_id, subfolder="scheduler")

### Optimizer

In [10]:
optimizer_cls = optim.AdamW

In [16]:
optimizer = optimizer_cls(
        unet.parameters(),
        lr=learning_rate,
        betas=(0.9, 0.999),  # default values
        weight_decay=1e-2,  #default value
        eps=1e-08,  #default value
)

## Prepare our trainingset